In [2]:
import requests
from lxml import html
import csv
import pandas as pd
import re

Extract all product detail infomation such as title, description, you should also handle pagination here so in the end you can get about 100+ records.

https://scrapingclub.com/exercise/list_basic/

### First page 

- First, scrape data (product title, product url, product price) from the main page.

In [3]:
r = requests.get('https://scrapingclub.com/exercise/list_basic/')
data = html.fromstring(r.text)

In [4]:
r = requests.get('https://scrapingclub.com/exercise/list_basic/')
data = html.fromstring(r.text)
# collect name, sentiment, date, review
alldata=[]
for review in data.xpath("//div[@class='card']"):
    title = review.xpath('div/h4/a/text()')
    url = review.xpath('a/@href')
    price= review.xpath('div/h5/text()') 
    print(title, url, price)
alldata.append([title, url, price])
    
len(alldata)

['Short Dress'] ['/exercise/list_basic_detail/90008-E/'] ['$24.99']
['Patterned Slacks'] ['/exercise/list_basic_detail/96436-A/'] ['$29.99']
['Short Chiffon Dress'] ['/exercise/list_basic_detail/93926-B/'] ['$49.99']
['Off-the-shoulder Dress'] ['/exercise/list_basic_detail/90882-B/'] ['$59.99']
['V-neck Top'] ['/exercise/list_basic_detail/93756-C/'] ['$24.99']
['Short Chiffon Dress'] ['/exercise/list_basic_detail/93926-C/'] ['$49.99']
['V-neck Top'] ['/exercise/list_basic_detail/93756-B/'] ['$24.99']
['V-neck Top'] ['/exercise/list_basic_detail/93756-D/'] ['$24.99']
['Short Lace Dress'] ['/exercise/list_basic_detail/96643-A/'] ['$59.99']


1

### Web Crawling

- Now, figure out the urls for the rest of web pages. Then, crawl a large data (100+ records) from all the pages

In [3]:
web = "https://scrapingclub.com/exercise/list_basic/?page=%s" 

# let's get the first four urls
for page in range(1,9):
    print(web % page)

https://scrapingclub.com/exercise/list_basic/?page=1
https://scrapingclub.com/exercise/list_basic/?page=2
https://scrapingclub.com/exercise/list_basic/?page=3
https://scrapingclub.com/exercise/list_basic/?page=4
https://scrapingclub.com/exercise/list_basic/?page=5
https://scrapingclub.com/exercise/list_basic/?page=6
https://scrapingclub.com/exercise/list_basic/?page=7
https://scrapingclub.com/exercise/list_basic/?page=8


In [4]:

web = "https://scrapingclub.com/exercise/list_basic/?page=%s" 
for page in range(1,8):
    url = web % page
    data = html.fromstring(requests.get(url).text)
    
    for review in data.xpath("//div[@class='card']"):
        title = review.xpath("div[@class='card-body']/h4/a/text()")
        urls = review.xpath("div[@class='card-body']/h4/a/@href")
        description = review.xpath("div[@class='card-body']/h5/text()")
        print(title, urls, description)
    

['Short Dress'] ['/exercise/list_basic_detail/90008-E/'] ['$24.99']
['Patterned Slacks'] ['/exercise/list_basic_detail/96436-A/'] ['$29.99']
['Short Chiffon Dress'] ['/exercise/list_basic_detail/93926-B/'] ['$49.99']
['Off-the-shoulder Dress'] ['/exercise/list_basic_detail/90882-B/'] ['$59.99']
['V-neck Top'] ['/exercise/list_basic_detail/93756-C/'] ['$24.99']
['Short Chiffon Dress'] ['/exercise/list_basic_detail/93926-C/'] ['$49.99']
['V-neck Top'] ['/exercise/list_basic_detail/93756-B/'] ['$24.99']
['V-neck Top'] ['/exercise/list_basic_detail/93756-D/'] ['$24.99']
['Short Lace Dress'] ['/exercise/list_basic_detail/96643-A/'] ['$59.99']
['Fitted Dress'] ['/exercise/list_basic_detail/94766-A/'] ['$34.99']
['V-neck Jumpsuit'] ['/exercise/list_basic_detail/91696-C/'] ['$69.99']
['Chiffon Dress'] ['/exercise/list_basic_detail/93745-A/'] ['$54.99']
['Skinny High Waist Jeans'] ['/exercise/list_basic_detail/65763-E/'] ['$39.99']
['Super Skinny High Jeans'] ['/exercise/list_basic_detail/70745

### Collect Detailed Product Description for Individual Product Webpages

- First, get **complete url** for each product

In [35]:
finaldata=[]

web = "https://scrapingclub.com/exercise/list_basic/?page=%s" 

for page in range(1,8):
    url = web % page
    data = html.fromstring(requests.get(url).text)
    
    for review in data.xpath("//div[@class='card']"):
        title = review.xpath("div[@class='card-body']/h4/a/text()")
        urls = review.xpath("div[@class='card-body']/h4/a/@href")
        description = review.xpath("div[@class='card-body']/h5/text()")   
        print(title, urls, description)
        finaldata.append([title, urls, description])
        
len(finaldata)

['Short Dress'] ['/exercise/list_basic_detail/90008-E/'] ['$24.99']
['Patterned Slacks'] ['/exercise/list_basic_detail/96436-A/'] ['$29.99']
['Short Chiffon Dress'] ['/exercise/list_basic_detail/93926-B/'] ['$49.99']
['Off-the-shoulder Dress'] ['/exercise/list_basic_detail/90882-B/'] ['$59.99']
['V-neck Top'] ['/exercise/list_basic_detail/93756-C/'] ['$24.99']
['Short Chiffon Dress'] ['/exercise/list_basic_detail/93926-C/'] ['$49.99']
['V-neck Top'] ['/exercise/list_basic_detail/93756-B/'] ['$24.99']
['V-neck Top'] ['/exercise/list_basic_detail/93756-D/'] ['$24.99']
['Short Lace Dress'] ['/exercise/list_basic_detail/96643-A/'] ['$59.99']
['Fitted Dress'] ['/exercise/list_basic_detail/94766-A/'] ['$34.99']
['V-neck Jumpsuit'] ['/exercise/list_basic_detail/91696-C/'] ['$69.99']
['Chiffon Dress'] ['/exercise/list_basic_detail/93745-A/'] ['$54.99']
['Skinny High Waist Jeans'] ['/exercise/list_basic_detail/65763-E/'] ['$39.99']
['Super Skinny High Jeans'] ['/exercise/list_basic_detail/70745

60

In [36]:
# convert to dataframe

# to view entire column
pd.set_option('display.max_colwidth', -1)
# convert list to data frame (excel-like)
df = pd.DataFrame(finaldata)
df.head(2)

,0,1,2
0,[Short Dress],[/exercise/list_basic_detail/90008-E/],[$24.99]
1,[Patterned Slacks],[/exercise/list_basic_detail/96436-A/],[$29.99]


In [37]:
# rename columns (title, url, price)

df.columns = ['title','url','price']
df.head(2)

,title,url,price
0,[Short Dress],[/exercise/list_basic_detail/90008-E/],[$24.99]
1,[Patterned Slacks],[/exercise/list_basic_detail/96436-A/],[$29.99]


In [38]:
# focus on the url column only and convert it to list

urls = df['url'].tolist()
for i in urls:
    print(i)

['/exercise/list_basic_detail/90008-E/']
['/exercise/list_basic_detail/96436-A/']
['/exercise/list_basic_detail/93926-B/']
['/exercise/list_basic_detail/90882-B/']
['/exercise/list_basic_detail/93756-C/']
['/exercise/list_basic_detail/93926-C/']
['/exercise/list_basic_detail/93756-B/']
['/exercise/list_basic_detail/93756-D/']
['/exercise/list_basic_detail/96643-A/']
['/exercise/list_basic_detail/94766-A/']
['/exercise/list_basic_detail/91696-C/']
['/exercise/list_basic_detail/93745-A/']
['/exercise/list_basic_detail/65763-E/']
['/exercise/list_basic_detail/70745-D/']
['/exercise/list_basic_detail/94706-A/']
['/exercise/list_basic_detail/96230-B/']
['/exercise/list_basic_detail/73840-F/']
['/exercise/list_basic_detail/65763-A/']
['/exercise/list_basic_detail/96230-A/']
['/exercise/list_basic_detail/73840-P/']
['/exercise/list_basic_detail/73840-Q/']
['/exercise/list_basic_detail/96113-C/']
['/exercise/list_basic_detail/96230-C/']
['/exercise/list_basic_detail/00959-A/']
['/exercise/list

In [41]:
#remove the extra bracket
urls = [x for y in urls for x in y]
urls

['/exercise/list_basic_detail/90008-E/',
 '/exercise/list_basic_detail/96436-A/',
 '/exercise/list_basic_detail/93926-B/',
 '/exercise/list_basic_detail/90882-B/',
 '/exercise/list_basic_detail/93756-C/',
 '/exercise/list_basic_detail/93926-C/',
 '/exercise/list_basic_detail/93756-B/',
 '/exercise/list_basic_detail/93756-D/',
 '/exercise/list_basic_detail/96643-A/',
 '/exercise/list_basic_detail/94766-A/',
 '/exercise/list_basic_detail/91696-C/',
 '/exercise/list_basic_detail/93745-A/',
 '/exercise/list_basic_detail/65763-E/',
 '/exercise/list_basic_detail/70745-D/',
 '/exercise/list_basic_detail/94706-A/',
 '/exercise/list_basic_detail/96230-B/',
 '/exercise/list_basic_detail/73840-F/',
 '/exercise/list_basic_detail/65763-A/',
 '/exercise/list_basic_detail/96230-A/',
 '/exercise/list_basic_detail/73840-P/',
 '/exercise/list_basic_detail/73840-Q/',
 '/exercise/list_basic_detail/96113-C/',
 '/exercise/list_basic_detail/96230-C/',
 '/exercise/list_basic_detail/00959-A/',
 '/exercise/list

In [42]:
# to test xpath, I will use the first url only
url = urls[0]
print(url)

/exercise/list_basic_detail/90008-E/


The above url is incomplete. So let's develop a complete url. It needs the domain name 'https://scrapingclub.com'

In [43]:
url = 'https://scrapingclub.com' + url
url

'https://scrapingclub.com/exercise/list_basic_detail/90008-E/'

In [44]:
r = requests.get(url)
data = html.fromstring(r.text)

In [45]:
proddesc = data.xpath("//p[@class='card-text']/text()")
print(proddesc)

['Short dress in woven fabric. Round neckline and opening at back of neck with a button. Yoke at back with concealed pleats, long sleeves, and narrow cuffs with ties. Side pockets. 100% polyester. Machine wash cold.']


Great! Everything works! Now, let's crawl more data.

In [46]:
# let's crawl the first ten urls (products) only

for i in urls[:10]:
    print(i)

/exercise/list_basic_detail/90008-E/
/exercise/list_basic_detail/96436-A/
/exercise/list_basic_detail/93926-B/
/exercise/list_basic_detail/90882-B/
/exercise/list_basic_detail/93756-C/
/exercise/list_basic_detail/93926-C/
/exercise/list_basic_detail/93756-B/
/exercise/list_basic_detail/93756-D/
/exercise/list_basic_detail/96643-A/
/exercise/list_basic_detail/94766-A/


In [47]:
review = []

# let's crawl the first ten urls only

for url in urls[:10]:
    r = requests.get('https://scrapingclub.com' + url)
    data = html.fromstring(r.text)
    
    proddesc = data.xpath("//p[@class='card-text']/text()")
    print(proddesc)

len(review)

['Short dress in woven fabric. Round neckline and opening at back of neck with a button. Yoke at back with concealed pleats, long sleeves, and narrow cuffs with ties. Side pockets. 100% polyester. Machine wash cold.']
['Ankle-length slacks in patterned stretch cotton satin. Regular waist with concealed hook-and-eye fastener and zip fly. Side pockets and tapered legs with slits at hems. 61% cotton, 36% polyester, 3% spandex. Machine wash...']
['Short V-neck dress in plumeti chiffon. Gathers and small ruffle at shoulders, dropped shoulders, and long, wide sleeves with buttons at cuffs. Narrow, elasticized seam at waist and circle skirt with ruffled tiers. Lined. 100% polyester. Machine wash warm.']
['Short, fitted off-the-shoulder dress in stretch ribbed fabric. Short raglan sleeves, seam at waist with gently flared skirt. Unlined. 54% rayon, 44% nylon, 2% spandex. Machine wash warm.']
['Top in woven fabric with V-neck front and back. Narrow shoulder straps and decorative straps at back.

0

In [48]:
alldesc = []

# let's crawl all the products

for url in urls:
    r = requests.get('https://scrapingclub.com' + url)
    data = html.fromstring(r.text)
    
    proddesc = data.xpath("//p[@class='card-text']/text()")
    print(proddesc)
    alldesc.append([proddesc])

len(alldesc)

['Short dress in woven fabric. Round neckline and opening at back of neck with a button. Yoke at back with concealed pleats, long sleeves, and narrow cuffs with ties. Side pockets. 100% polyester. Machine wash cold.']
['Ankle-length slacks in patterned stretch cotton satin. Regular waist with concealed hook-and-eye fastener and zip fly. Side pockets and tapered legs with slits at hems. 61% cotton, 36% polyester, 3% spandex. Machine wash...']
['Short V-neck dress in plumeti chiffon. Gathers and small ruffle at shoulders, dropped shoulders, and long, wide sleeves with buttons at cuffs. Narrow, elasticized seam at waist and circle skirt with ruffled tiers. Lined. 100% polyester. Machine wash warm.']
['Short, fitted off-the-shoulder dress in stretch ribbed fabric. Short raglan sleeves, seam at waist with gently flared skirt. Unlined. 54% rayon, 44% nylon, 2% spandex. Machine wash warm.']
['Top in woven fabric with V-neck front and back. Narrow shoulder straps and decorative straps at back.

60

### Combine Project Information with Product Description

In [49]:
# how many products?
len(df)


60

In [50]:
# how many reviews?
len(alldesc)


60

for joining data, we need dataframes.

In [51]:
# convert alldesc to dataframe
df_alldesc = pd.DataFrame(alldesc)
df_alldesc.head()

,0
0,"[Short dress in woven fabric. Round neckline and opening at back of neck with a button. Yoke at back with concealed pleats, long sleeves, and narrow cuffs with ties. Side pockets. 100% polyester. Machine wash cold.]"
1,"[Ankle-length slacks in patterned stretch cotton satin. Regular waist with concealed hook-and-eye fastener and zip fly. Side pockets and tapered legs with slits at hems. 61% cotton, 36% polyester, 3% spandex. Machine wash...]"
2,"[Short V-neck dress in plumeti chiffon. Gathers and small ruffle at shoulders, dropped shoulders, and long, wide sleeves with buttons at cuffs. Narrow, elasticized seam at waist and circle skirt with ruffled tiers. Lined. 100% polyester. Machine wash warm.]"
3,"[Short, fitted off-the-shoulder dress in stretch ribbed fabric. Short raglan sleeves, seam at waist with gently flared skirt. Unlined. 54% rayon, 44% nylon, 2% spandex. Machine wash warm.]"
4,[Top in woven fabric with V-neck front and back. Narrow shoulder straps and decorative straps at back. 100% polyester. Machine wash cold.]


In [52]:
# join two dataframes and create a large dataframe
df_alldata = df.join(df_alldesc)
df_alldata

,title,url,price,0
0,[Short Dress],[/exercise/list_basic_detail/90008-E/],[$24.99],"[Short dress in woven fabric. Round neckline and opening at back of neck with a button. Yoke at back with concealed pleats, long sleeves, and narrow cuffs with ties. Side pockets. 100% polyester. Machine wash cold.]"
1,[Patterned Slacks],[/exercise/list_basic_detail/96436-A/],[$29.99],"[Ankle-length slacks in patterned stretch cotton satin. Regular waist with concealed hook-and-eye fastener and zip fly. Side pockets and tapered legs with slits at hems. 61% cotton, 36% polyester, 3% spandex. Machine wash...]"
2,[Short Chiffon Dress],[/exercise/list_basic_detail/93926-B/],[$49.99],"[Short V-neck dress in plumeti chiffon. Gathers and small ruffle at shoulders, dropped shoulders, and long, wide sleeves with buttons at cuffs. Narrow, elasticized seam at waist and circle skirt with ruffled tiers. Lined. 100% polyester. Machine wash warm.]"
3,[Off-the-shoulder Dress],[/exercise/list_basic_detail/90882-B/],[$59.99],"[Short, fitted off-the-shoulder dress in stretch ribbed fabric. Short raglan sleeves, seam at waist with gently flared skirt. Unlined. 54% rayon, 44% nylon, 2% spandex. Machine wash warm.]"
4,[V-neck Top],[/exercise/list_basic_detail/93756-C/],[$24.99],[Top in woven fabric with V-neck front and back. Narrow shoulder straps and decorative straps at back. 100% polyester. Machine wash cold.]
5,[Short Chiffon Dress],[/exercise/list_basic_detail/93926-C/],[$49.99],"[Short V-neck dress in plumeti chiffon. Gathers and small ruffle at shoulders, dropped shoulders, and long, wide sleeves with buttons at cuffs. Narrow, elasticized seam at waist and circle skirt with ruffled tiers. Lined. 100% polyester. Machine wash warm.]"
6,[V-neck Top],[/exercise/list_basic_detail/93756-B/],[$24.99],[Top in woven fabric with V-neck front and back. Narrow shoulder straps and decorative straps at back. 100% polyester. Machine wash cold.]
7,[V-neck Top],[/exercise/list_basic_detail/93756-D/],[$24.99],[Top in woven fabric with V-neck front and back. Narrow shoulder straps and decorative straps at back. 100% polyester. Machine wash cold.]
8,[Short Lace Dress],[/exercise/list_basic_detail/96643-A/],[$59.99],"[Short, straight-cut dress in lace. Opening at back with concealed button at back of neck. Round neck, padded shoulders, and 3/4-length sleeves. Lined. 51% cotton, 25% nylon, 24% rayon. Machine wash cold.]"
9,[Fitted Dress],[/exercise/list_basic_detail/94766-A/],[$34.99],"[Fitted dress in thick jersey with a V-neck, long sleeves, and cut-out section at back with button at back of neck. Lined at top. 60% rayon, 35% nylon, 5% spandex. Machine wash warm.]"
